In [117]:
import sympy as sym
import numpy as np
import math
import scipy as sp

In [118]:
#Equation Setup
la,wa,ta,E,G,J,lb,wb,tb,lc,wc,tc,l,w,t,psi,d = sym.symbols('la,wa,ta,E,G,J,lb,wb,tb,lc,wc,tc,l,w,t,psi,d')

Ca = sym.Matrix([[0, 0, 0, la/(G*J), 0, 0],
             [0, 0, -6*la**2/(ta*wa**3*E), 0, 12*la/(ta*wa**3*E), 0],
             [0, 6*la**2/(ta**3*wa*E), 0, 0, 0, 12*la/(ta**3*wa*E)],
             [la/(ta*wa*E), 0, 0, 0, 0, 0],
             [0, 4*la**3/(ta**3*wa*E), 0, 0, 0, 6*la**2/(ta**3*wa*E)],
             [0, 0, 4*la**3/(ta*wa**3*E), 0, -6*la**2/(ta*wa**3*E), 0]])

Cb = sym.Matrix([[0, 0, 0, lb/(G*J), 0, 0],
             [0, 0, -6*lb**2/(wb*tb**3*E), 0, 12*lb/(wb*tb**3*E), 0],
             [0, 6*lb**2/(wb**3*tb*E), 0, 0, 0, 12*lb/(wb**3*tb*E)],
             [lb/(wb*tb*E), 0, 0, 0, 0, 0],
             [0, 4*lb**3/(wb**3*tb*E), 0, 0, 0, 6*lb**2/(wb**3*tb*E)],
             [0, 0, 4*lb**3/(wb*tb**3*E), 0, -6*lb**2/(wb*tb**3*E), 0]])

Cc = sym.Matrix([[0, 0, 0, lc/(G*J), 0, 0],
             [0, 0, -6*lc**2/(tc*wc**3*E), 0, 12*lc/(tc*wc**3*E), 0],
             [0, 6*lc**2/(tc**3*wc*E), 0, 0, 0, 12*lc/(tc**3*wc*E)],
             [lc/(tc*wc*E), 0, 0, 0, 0, 0],
             [0, 4*lc**3/(tc**3*wc*E), 0, 0, 0, 6*lc**2/(tc**3*wc*E)],
             [0, 0, 4*lc**3/(tc*wc**3*E), 0, -6*lc**2/(tc*wc**3*E), 0]])

R = sym.eye(3)

Da = sym.Matrix([[0,0,0],
             [0,0, lb+lc],
             [0,-lb-lc,0]])

Db = sym.Matrix([[0,0,0],
             [0,0,lc],
             [0,-lc,0]])

Ad_a = sym.Matrix([[R, sym.zeros(3)],
               [Da*R, R]])

Ad_b = sym.Matrix([[R, sym.zeros(3)],
               [Db*R, R]])

Ct = Ad_a*Ca*Ad_a.inv()+Ad_b*Cb*Ad_b.inv()+Cc
Ct_Assumed = sym.simplify(Ct.subs({ta:t,tb:t,tc:t,wa:w,wb:w,wc:w,la:l,lb:l,lc:l}))

R1 = sym.Matrix([[sym.cos((sym.pi+psi)/2), -sym.sin((sym.pi+psi)/2), 0],
             [sym.sin((sym.pi+psi)/2), sym.cos((sym.pi+psi)/2), 0],
             [0, 0, 1]])
#display(R1)

R2 = sym.Matrix([[sym.cos((sym.pi-psi)/2), -sym.sin((sym.pi-psi)/2), 0],
             [sym.sin((sym.pi-psi)/2), sym.cos((sym.pi-psi)/2), 0],
             [0, 0, 1]])
#display(R2)

D1 = sym.Matrix([[0, 0, 0],
             [0, 0, -d/2],
             [0, d/2, 0]])
#display(D1)

D2 = sym.Matrix([[0, 0, 0],
             [0, 0, d/2],
             [0, -d/2, 0]])
#display(D2)


Ad_1 = sym.Matrix([[R1, sym.zeros(3)],
               [D1*R1, R1]])
#display(Ad_1)

Ad_2 = sym.Matrix([[R2, sym.zeros(3)],
               [D2*R2, R2]])
#display(Ad_2)

Kt_Assumed = sym.cancel(Ct_Assumed.inv())
Kp_Assumed = sym.cancel(Ad_1*Kt_Assumed*Ad_1.inv() + Ad_2*Kt_Assumed*Ad_2.inv())

#Abbreviating Elements for Display
m, n = Kp_Assumed.shape
C = sym.Matrix(m, n, lambda i, j: sym.Symbol(f'k{i+1}{j+1}'))

# Loop through each element and replace non-zero entries with placeholders
for i in range(m):
    for j in range(n):
        if Kp_Assumed[i, j] == 0:
            C[i, j] = 0  # Retain sym.zeros

display(C)

Matrix([
[  0,   0, k13, k14,   0,   0],
[  0,   0,   0,   0, k25,   0],
[k31,   0,   0,   0,   0, k36],
[k41,   0,   0,   0,   0, k46],
[  0, k52,   0,   0,   0,   0],
[  0,   0, k63, k64,   0,   0]])

In [119]:
#Calculation
Load = sym.Matrix([150,-588.6,150,-88.29,0,0])
t0,w0,l0,d0,psi0,E0,G0 = 3e-3,20e-3,20e-3,20e-3,math.radians(70),125e9,48e9
J0 = 0.312*w0*t0**3
Volume = t0*w0*l0*3*2 + t0*w0**2*4 + w0*d0**2
Mass = 4480*Volume
y_size = (3*l0+t0*4)*sym.cos(psi0/2)+d0/2
x_size = (3*l0+t0*3)*sym.sin(psi0/2)*2+d0
z_size = w0

Kp_Assumed_num = Kp_Assumed.subs({t:t0, w:w0, l:l0, d:d0, psi:psi0, E:E0, G:G0, J:J0})
Cp_Assumed_num = Kp_Assumed_num.inv()
freq_x = float(1/(2*sym.pi*sym.sqrt(Mass*Cp_Assumed_num[4-1,1-1])))
freq_z = float(1/(2*sym.pi*sym.sqrt(Mass*Cp_Assumed_num[6-1,3-1])))
Kp_Assumed_numpy = np.array(Kp_Assumed_num).astype(np.float64)
#eig_val = np.array(list(Kp_Assumed_num.eigenvals().keys()))
eig_val, eig_vec = np.linalg.eig(Kp_Assumed_numpy)
#display(Kp_Assumed_numpy)
#display(eig_vec)
#Kp_Assumed_num.eigenvects()[i][2][0] for i in range(3))
#display(type(eig_val))
#display(np.array(list(eig_val.keys())))
#display(1/(2*np.pi)*np.sqrt(abs(eig_val)))

Displacement = Kp_Assumed_num.solve(Load)
#Display Values
print(f"Mass = {Mass*1000:.2f} g")
print(f"X Size = {x_size*1000:.2f} mm\nY Size = {y_size*1000:.2f} mm\nZ Size = {z_size*1000:.2f} mm")
print(f"Natural Frequency in X Axis = {freq_x:.2f} Hz\nNatural Frequency in Z Axis = {freq_z:.2f} Hz")
print(f"X Displacement = {Displacement[3]*1000:.4f} mm\nY Displacement = {Displacement[4]*1000:.4f} mm\nZ Displacement = {Displacement[5]*1000:.4f} mm\nX Rotation = {math.degrees(Displacement[0]):.4f} deg\nY Rotation = {math.degrees(Displacement[1]):.4f} deg\nZ Rotation = {math.degrees(Displacement[2]):.4f} deg")

Mass = 89.60 g
X Size = 99.15 mm
Y Size = 68.98 mm
Z Size = 20.00 mm
Natural Frequency in X Axis = 1503.61 Hz
Natural Frequency in Z Axis = 443.65 Hz
X Displacement = 0.0188 mm
Y Displacement = -0.0035 mm
Z Displacement = -4.6074 mm
X Rotation = -10.7751 deg
Y Rotation = 0.0000 deg
Z Rotation = 0.0695 deg


In [120]:
# Assuming you have already defined Kp_Assumed, E0, G0, etc.

# Objective function to minimize mass
def objective_minimize_mass(params):
    w0, t0, l0, psi0, d0 = (np.array(params)*np.array(ub)).tolist()
    w1, t1, l1, psi1, d1 = ub
    Volume_ref =  t1 * w1 * l1 * 3 * 2 + t1 * w1**2 * 4 + w1 * d1**2
    Mass_ref = 4480 * Volume_ref
    # Calculate volume and mass
    Volume = t0 * w0 * l0 * 3 * 2 + t0 * w0**2 * 4 + w0 * d0**2
    Mass = 4480 * Volume
    Volume_norm = Volume/Volume_ref
    Mass_norm = Mass/Mass_ref
    
    # Calculate sizes
    x_size = (3 * l0 + t0 * 3) * math.sin(psi0 / 2) * 2 + d0
    x_ref = (3 * l1 + t1 * 3) * math.sin(psi1 / 2) * 2 + d1
    y_size = (3 * l0 + t0 * 4) * math.cos(psi0 / 2) + d0 / 2
    y_ref = (3 * l1 + t1 * 4) * math.cos(psi1 / 2) + d1 / 2
    z_size = w0
    z_ref = w1
    
    # Calculate frequency in X axis
    J0 = 0.312 * w0 * t0**3
    J1 = 0.312 * w1 * t1**3
    Kp_num = Kp_Assumed.subs({t: t0, w: w0, l: l0, d: d0, psi: psi0, E: E0, G: G0, J: J0})
    Kp_num_ref = Kp_Assumed.subs({t: t1, w: w1, l: l1, d: d1, psi: psi1, E: E0, G: G0, J: J1})
    Cp_num = Kp_num.inv()
    Cp_num_ref = Kp_num_ref.inv()
    Cp_num_x = Cp_num[4-1,1-1]
    Cp_num_x_ref = Cp_num_ref[4-1,1-1]
    freq_x = float(1 / (2 * math.pi * math.sqrt(Mass * Cp_num_x)))
    freq_x_ref = float(1 / (2 * math.pi * math.sqrt(Mass_ref * Cp_num_x_ref)))
    freq_x_norm = freq_x/freq_x_ref
    Cp_num_x_norm = (Cp_num_x/Cp_num_x_ref)**-1
    
    # Calculate frequency in Z axis
    Cp_num_z = Cp_num[6-1,3-1]
    Cp_num_z_ref = Cp_num_ref[6-1,3-1]
    freq_z = float(1 / (2 * math.pi * math.sqrt(Mass * Cp_num_z)))
    freq_z_ref = float(1 / (2 * math.pi * math.sqrt(Mass_ref * Cp_num_z_ref)))
    freq_z_norm = freq_z/freq_z_ref
    Cp_num_z_norm = (Cp_num_z/Cp_num_z_ref)**-1
    
    # Calculate parasitic error
    parasitic_error = Cp_num[4-1,6-1]/Cp_num[3-1,6-1]-(d0/math.tan(psi0/2))/2
    parasitic_error_ref = Cp_num_ref[4-1,6-1]/Cp_num_ref[3-1,6-1]-(d1/math.tan(psi1/2))/2
    parasitic_error_norm = (parasitic_error/parasitic_error_ref)**-1
    
    # Combined objective to minimize mass and maximize sizes
    combined_value = weight[0]*Mass_norm - weight[1]*(x_size/x_ref + y_size/y_ref + z_size/z_ref) + weight[2]*Cp_num_x_norm + weight[3]*Cp_num_z_norm + weight[4]*freq_x_norm + weight[5]*freq_z_norm + weight[6]*parasitic_error_norm
    
    return combined_value

# Constraint functions to ensure sizes are maximized
def constraint_x_size(params):
    w0, t0, l0, psi0, d0 = (np.array(params)*np.array(ub)).tolist()
    x_size = float((3 * l0 + t0 * 3) * math.sin(psi0 / 2) * 2 + d0)
    return float(x_size_max - x_size) # Ensure x size is below the maximum limit

def constraint_y_size(params):
    w0, t0, l0, psi0, d0 = (np.array(params)*np.array(ub)).tolist()
    y_size = float((3 * l0 + t0 * 4) * math.cos(psi0 / 2) + d0 / 2)
    return float(y_size_max - y_size)  # Ensure y size is below the maximum limit

def constraint_z_size(params):
    w0, t0, l0, psi0, d0 = (np.array(params)*np.array(ub)).tolist()
    z_size = float(w0)
    return float(z_size_max - z_size)  # Ensure z size is below the maximum limit

# Frequency constraints to ensure structural integrity
def constraint_freq_x(params):
    w0, t0, l0, psi0, d0 = (np.array(params)*np.array(ub)).tolist()
    J0 = 0.312 * w0 * t0**3
    Kp_num = Kp_Assumed.subs({t: t0, w: w0, l: l0, d: d0, psi: psi0, E: E0, G: G0, J: J0})
    Cp_num = Kp_num.inv()
    Volume = t0 * w0 * l0 * 3 * 2 + t0 * w0**2 * 4 + w0 * d0**2
    Mass = 4480 * Volume
    freq_x = float(1 / (2 * math.pi * math.sqrt(Mass * Cp_num[4-1, 1-1])))
    return freq_x - 200  # Ensure natural frequency in x is greater than 200 Hz

def constraint_freq_z(params):
    w0, t0, l0, psi0, d0 = (np.array(params)*np.array(ub)).tolist()
    J0 = 0.312 * w0 * t0**3
    Kp_num = Kp_Assumed.subs({t: t0, w: w0, l: l0, d: d0, psi: psi0, E: E0, G: G0, J: J0})
    Cp_num = Kp_num.inv()
    Volume = t0 * w0 * l0 * 3 * 2 + t0 * w0**2 * 4 + w0 * d0**2
    Mass = 4480 * Volume
    freq_z = float(1 / (2 * math.pi * math.sqrt(Mass * Cp_num[6-1, 3-1])))
    return freq_z - 200  # Ensure natural frequency in z is greater than 200 Hz

# Define the maximum allowed sizes in x, y, z directions
x_size_max = 0.100  # Example: 100 mm
y_size_max = 0.065  # Example: 100 mm
z_size_max = 0.030  # Example: 100 mm

# Initial guesses
initial_guess = [0.02, 0.003, 0.05, math.radians(60), 0.02]  # Example initial guesses for w, t, l, psi, d0
weight = [1, 0.5, 1, 1, 1, 1, 1] # For Mass, Size, Cp_x, Cp_z, Freq_X, Freq_Z, Parasitic Error
  
# Bounds for parameters using scipy's Bounds class
ub = [0.15, 0.010, 0.15, math.radians(100), 0.05]
lb = [0.010, 0.001, 0.010, math.radians(30), 0.01]
lb_norm = (np.array(lb)/np.array(ub)).tolist()
initial_guess_norm = (np.array(initial_guess)/np.array(ub)).tolist() # Initial Guesses normalized
#bounds = sp.optimize.Bounds([0.010, 0.001, 0.010, math.radians(30), 0.01], [0.15, 0.010, 0.15, math.radians(100), 0.05], True)
bounds = sp.optimize.Bounds(lb_norm, [1, 1, 1, 1, 1], True)

# Constraints for minimize function
constraints = [
    {'type': 'ineq', 'fun': constraint_x_size},
    {'type': 'ineq', 'fun': constraint_y_size},
    {'type': 'ineq', 'fun': constraint_z_size},
    {'type': 'ineq', 'fun': constraint_freq_x},
    {'type': 'ineq', 'fun': constraint_freq_z}
]

# Perform Basin Hopping
# Algorithms: SLSQP, Powell, L-BFGS-B, COBYLA, COBYQA, stepsize = 0.02, T = 0.2
minimizer_kwargs = {"method": "SLSQP", "bounds": bounds, "constraints": constraints}
print(initial_guess_norm)
result = sp.optimize.basinhopping(objective_minimize_mass, initial_guess_norm, minimizer_kwargs=minimizer_kwargs, niter=10, disp = True, stepsize = 0.1, T = 10)

if result.success:
    print('Optimization is successful')
else:
    print('Optimization failed')
optimized_params = result.x


[0.13333333333333333, 0.3, 0.33333333333333337, 0.6, 0.39999999999999997]


KeyboardInterrupt: 

In [116]:
# Function to calculate mass, natural frequencies, and sizes
def calculate_properties(params):
    w0, t0, l0, psi0, d0 = (np.array(params)*np.array(ub)).tolist()
    # Calculate mass
    Volume = t0 * w0 * l0 * 3 * 2 + t0 * w0**2 * 4 + w0 * d0**2
    Mass = 4480 * Volume
    
    # Calculate natural frequencies
    J0 = 0.312 * w0 * t0**3
    Kp_num = Kp_Assumed.subs({t: t0, w: w0, l: l0, d: d0, psi: psi0, E: E0, G: G0, J: J0})
    Cp_num = Kp_num.inv()
    freq_x = float(1 / (2 * math.pi * math.sqrt(Mass * Cp_num[4-1, 1-1])))
    freq_z = float(1 / (2 * math.pi * math.sqrt(Mass * Cp_num[6-1, 3-1])))
    
    # Calculate sizes
    x_size = float((3 * l0 + t0 * 3) * math.sin(psi0 / 2) * 2 + d0)
    y_size = float((3 * l0 + t0 * 4) * math.cos(psi0 / 2) + d0 / 2)
    z_size = float(w0)
    
    Cp_63 = float(Cp_num[6-1, 3-1])
    Cp_46 = float(Cp_num[4-1, 6-1])
    Parasitic_Error = float(Cp_num[4-1, 6-1]/Cp_num[3-1,6-1])-(d0/math.tan(psi0/2))/2
    
    Displacement = Kp_num.solve(Load)
    
    return Mass, freq_x, freq_z, x_size, y_size, z_size, Cp_63, Cp_46, Parasitic_Error, Displacement

optimized_mass, optimized_freq_x, optimized_freq_z, optimized_x_size, optimized_y_size, optimized_z_size, optimized_cp63, optimized_cp46, optimized_parasitic_error, optimized_displacement = calculate_properties(optimized_params)

optimized_params_abs = (np.array(optimized_params)*np.array(ub)).tolist()
print("\nOptimization Converged!")
print(f"Optimization Function Value = {objective_minimize_mass(optimized_params)} (dimensionless)")
print(f"Optimized Width (w): {optimized_params_abs[0]*1000:.2f} mm")
print(f"Optimized Thickness (t): {optimized_params_abs[1]*1000:.2f} mm")
print(f"Optimized Length (l): {optimized_params_abs[2]*1000:.2f} mm")
print(f"Optimized Angle (psi): {math.degrees(optimized_params_abs[3]):.2f} degrees")
print(f"Optimized Base Distance (d0): {optimized_params_abs[4]*1000:.2f} mm")
print(f"Optimized Mass: {optimized_mass*1000:.2f} g")
print(f"Natural Frequency in X Direction: {optimized_freq_x:.2f} Hz")
print(f"Natural Frequency in Z Direction: {optimized_freq_z:.2f} Hz")
print(f"Optimized Size X: {optimized_x_size*1000:.2f} mm")
print(f"Optimized Size Y: {optimized_y_size*1000:.2f} mm")
print(f"Optimized Size Z: {optimized_z_size*1000:.2f} mm")
print(f"Optimized Cp63: {optimized_cp63}")
print(f"Optimized Cp46: {optimized_cp46}")
print(f"Optimized Parasitic Error: {optimized_parasitic_error} mm")

print(f"\nOptimized Displacement in X Direction: {optimized_displacement[3]*1000:.4f} mm")
print(f"Optimized Displacement in Y Direction: {optimized_displacement[4]*1000:.4f} mm")
print(f"Optimized Displacement in Z Direction: {optimized_displacement[5]*1000:.4f} mm")
print(f"Optimized Rotation around X Axis: {math.degrees(optimized_displacement[0]):.4f} deg")
print(f"Optimized Rotation around Y Axis: {math.degrees(optimized_displacement[1]):.4f} deg")
print(f"Optimized Rotation around Z Axis: {math.degrees(optimized_displacement[2]):.4f} deg")

print(f"Optimized Params: {optimized_params}")



Optimization Converged!
Optimization Function Value = 6.09023966848187 (dimensionless)
Optimized Width (w): 10.00 mm
Optimized Thickness (t): 1.01 mm
Optimized Length (l): 10.00 mm
Optimized Angle (psi): 30.00 degrees
Optimized Base Distance (d0): 34.46 mm
Optimized Mass: 57.69 g
Natural Frequency in X Direction: 264.53 Hz
Natural Frequency in Z Direction: 200.00 Hz
Optimized Size X: 51.55 mm
Optimized Size Y: 50.09 mm
Optimized Size Z: 10.00 mm
Optimized Cp63: 1.097604969868484e-05
Optimized Cp46: 9.688401491557816e-05
Optimized Parasitic Error: -0.0013489383354391288 mm

Optimized Displacement in X Direction: 0.9411 mm
Optimized Displacement in Y Direction: -0.0075 mm
Optimized Displacement in Z Direction: -62.2077 mm
Optimized Rotation around X Axis: -246.2931 deg
Optimized Rotation around Y Axis: 0.0000 deg
Optimized Rotation around Z Axis: 0.8327 deg
Optimized Params: [0.06666667 0.10052464 0.06666667 0.3        0.68914396]
